In [1]:
link='https://borrowell.com/?msclkid=7db7376a9d6e1045e14650cc9356e94b'

In [42]:
!pip install -q langchain-openai chromadb langchain langchain_core beautifulsoup4 langchain_community faiss-cpu

In [3]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ['LANGCHAIN_TRACING_V2']="true"
os.environ['LANGCHAIN_PROJECT']=os.getenv('LANGCHAIN_PROJECT')

In [1]:
#Data Ingestion 
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [11]:
loader=WebBaseLoader(link)

In [12]:
loader

In [14]:
web_docs=loader.load()
print(web_docs[:10])

[Document(metadata={'source': 'https://borrowell.com/?msclkid=7db7376a9d6e1045e14650cc9356e94b', 'title': 'Credit Report & Credit Score Canada | Borrowell™', 'description': "1 in 10 Canadians use Borrowell to monitor their credit. No credit card or SIN needed. We're the only free credit score app made in Canada.", 'language': 'en'}, page_content="Credit Report & Credit Score Canada | Borrowell™Log InMenuLog InSign UpFree Credit ScoreRent AdvantageCredit BuilderCredit CardsPersonal LoansMortgagesBankingInsuranceBlogStart With Your Free Credit ReportJoin 3 million members and access your free credit score. Get personalized tips, tools and offers that fit your profile and help you meet your financial goals.Sign Up for Free3 million happy Borrowell members.Borrowell is proudly Canadian and was first to offer free credit scores in Canada.Top 5 Reasons Why Canadians Love BorrowellAs Canada's leading credit education company we offer more than just free credit scores!Free weekly credit score 

In [18]:
#chunking -->
#covert to vector with vector embedding
#--> store in vectordb

from langchain_text_splitters  import RecursiveCharacterTextSplitter
splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
doc_split=splitter.split_documents(web_docs)

In [33]:
#doc_split
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
embed=OpenAIEmbeddings()
vecstorstoredb=FAISS.from_documents(doc_split, embed)
vecstorstoredb.save_local('faiss_embed_website')

In [36]:
query="what is a credit score? and how can i track it?"
result=vecstorstoredb.similarity_search(query)

In [38]:
result[0]

Document(id='4b15973b-ef46-4998-8ac4-f502c82b77a4', metadata={'source': 'https://borrowell.com/?msclkid=7db7376a9d6e1045e14650cc9356e94b', 'title': 'Credit Report & Credit Score Canada | Borrowell™', 'description': "1 in 10 Canadians use Borrowell to monitor their credit. No credit card or SIN needed. We're the only free credit score app made in Canada.", 'language': 'en'}, page_content="Coach helps you understand your credit score and gives personalized tips that may help you improve it.Product RecommendationsCompare financial products from over 75+ partners and get personalized recommendations based on your credit profile.Get StartedYour Credit Score and Report - All in One Place!You can get your Equifax credit score and credit report for free with Borrowell! Check your credit score or download your credit report whenever you'd like. Receive weekly updates on how your score")

In [53]:
#Retrieval chain ask a question that is more meaningful
#Document Chain
#RAG
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template(
    """
    Answer the following question based on the provided context:
    {context}
    """
) 
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model='gpt-4o')



In [54]:
document_chain=create_stuff_documents_chain(llm, prompt)

In [55]:
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following question based on the provided context:\n    {context}\n    '), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f5e852dfe50>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f5e881b2090>, root_client=<openai.OpenAI object at 0x7f5ea82b6050>, root_async_client=<openai.AsyncOpenAI object at 0x7f5e9b783ed0>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[]

In [57]:
from langchain_core.documents import Document
document_chain.invoke(
    {
        "input": "How can i crease my credit score?",
        "context":[Document(page_content="Your credit score can have a big impact on your financial future. Sign up for Borrowell to get your credit score and credit report for free! Join over 3 million Canadians and get the tools you need to help understand, manage, and master your credit - in under 3 minutes. Checking your credit score with Borrowell won't hurt your score.")]
    }
)

'What is Borrowell, and how does it benefit its users?\n\nBorrowell is a service that provides users with access to their credit score and credit report at no cost. It aims to help users understand, manage, and master their credit. The platform is used by over 3 million Canadians. Additionally, checking your credit score with Borrowell does not negatively impact your score, making it a convenient option for monitoring credit health.'

In [61]:
#however we want the document to first come from the retriever
#Retriever --->
#Input --> Retriever(just like an interface)--> Vectorestoredb

retriever=vecstorstoredb.as_retriever()

from langchain.chains import create_retrieval_chain

retrieval_chain=create_retrieval_chain(retriever, document_chain)

In [62]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f5ea806e250>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following question based on the provided context:\n    {context}\n    '), additional_kwargs={})])
            | ChatOpenAI(client=<opena

In [65]:
## Get the response from the LLM
response=retrieval_chain.invoke(
    {"input": "How can i sign up to see my credit score?"})

In [66]:
response['answer']

'Based on the context provided, here are the main features and services offered by Borrowell:\n\n1. **Free Credit Score and Report**: Borrowell allows you to access your Equifax credit score and report for free. You can check and download your credit report whenever you like without affecting your credit score.\n\n2. **Weekly Updates**: You receive weekly updates on changes to your credit score.\n\n3. **Personalized Tips**: Borrowell provides personalized advice on improving your credit score through their AI-powered Credit Coach.\n\n4. **Product Recommendations**: You can compare financial products from over 75 partners. Borrowell offers personalized recommendations based on your credit profile for things like loans, credit cards, and insurance.\n\n5. **Free Weekly Credit Monitoring**: Borrowell updates your Equifax credit score and report weekly, and it’s all free after signing up.\n\n6. **Credit Education**: The platform includes educational content to help you understand and manage

In [69]:
response['context']

[Document(id='4accc813-4c84-4ae7-b368-6ec33baa8ee6', metadata={'source': 'https://borrowell.com/?msclkid=7db7376a9d6e1045e14650cc9356e94b', 'title': 'Credit Report & Credit Score Canada | Borrowell™', 'description': "1 in 10 Canadians use Borrowell to monitor their credit. No credit card or SIN needed. We're the only free credit score app made in Canada.", 'language': 'en'}, page_content="score or download your credit report whenever you'd like. Receive weekly updates on how your score has changed, get personalized tips how to improve your score, and find the best financial products that match your profile. Checking your credit score with Borrowell doesn't hurt it.Get Your Credit ScoreIs All This Really Free?Yes, it's really free."),
 Document(id='4b15973b-ef46-4998-8ac4-f502c82b77a4', metadata={'source': 'https://borrowell.com/?msclkid=7db7376a9d6e1045e14650cc9356e94b', 'title': 'Credit Report & Credit Score Canada | Borrowell™', 'description': "1 in 10 Canadians use Borrowell to moni

In [53]:
from langchain_core.prompts import ChatPromptTemplate
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
load_dotenv()
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
from langchain_core.output_parsers import StrOutputParser

from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model='gpt-4o')
parser=StrOutputParser()
prompt=ChatPromptTemplate.from_messages(
    [
   ('system',"""
   Your an expert in finding check numbers and check amounts, you excel in understanding the context of the comments,
   Extract the check number and the ammount associated with the check based 
   on the provided user input return a dictionary with key value pair if no 
   amount exists you can have the check no and put null, but please be consistent and have grounded response, do not answer anything extra"""),
    ('user','{input}')
    ]
)


In [54]:

comment_test = """
Balabommala Srinivasa Rao (OFFICE\bsrin05), Thu Dec 19 2024 07:46:23 GMT-0500 (Eastern Standard Time)Date of Narrative entry: 12/19/2024Claim Log :# 28033Claim total amount:$ 10342.66Account Number#: 168065175Claim (Accepted/Denied/In progress):- In progressCustomer Letter Sent Y/N:- NBanker/Internal Contact: ashley.reckers@bmo.comAccount Status: 09-Credits Only AllowedRestriction or Closure submitted: (Y/N):- Ni.e.) from (ashley.reckers@bmo.com) on behalf of (2012 PATRICK M MACAULAY REVOCABLE TRUST) in regard to check fraud that occurred on (12/04/2024) with check number (3600) originally payable to (US Bank) which was altered by (Shaquille Hilliard) transacted at (BMO Bank). The claim states that the check is (Check Alterations). Additional information was provided on how the check may have been compromised. Details are as follows: sent 1st letter to BOFD, it&rsquo;s a sec-2 claim.12/04       3600 15488 51516 0093         10342.66 CK OTC CHECKBUC : 0278912/19/2024: - Email sent to manager for approval to do
"""


In [55]:
chain=prompt|llm| parser
response=chain.invoke({"input":comment_test})

In [56]:
import json
response=response.strip("```json\n").strip("\n```")
check_amount_dict=json.loads(response)

In [57]:
check_amount_dict

{'3600': 10342.66}